In [1]:
import os 


In [4]:
!pwd

/Users/sanjaymahto/Chess-Cancer-Classification/Chest-Cancer-Project


In [3]:
os.chdir('../')

In [ ]:
export MLFLOW_TRACKING_URI=https://dagshub.com/maushamkumar/Chest-Cancer-Project.mlflow

export MLFLOW_TRACKING_USERNAME=Mausham-Kumar

export MLFLOW_TRACKING_PASSWORD=d9d8c948482916cbef26519e40e72a2888bacbc0

In [5]:
os.environ['MLFLOW_TRACKING_URI']='https://dagshub.com/maushamkumar/Chest-Cancer-Project.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME']='Mausham-Kumar'
os.environ['MLFLOW_TRACKING_PASSWORD']='d9d8c948482916cbef26519e40e72a2888bacbc0'

In [5]:
import tensorflow as tf

In [6]:
model = tf.keras.models.load_model('artifacts/training/model.h5')

In [7]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [8]:
from src.CNN.constants import *
from src.CNN.utils.common import  read_yaml, create_directories, save_json

In [9]:

class ConfigurationManager:
    def __init__(self, 
                 config_filepath = CONFIG_FILE_PATH, 
                 params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
        
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri='https://dagshub.com/maushamkumar/Chest-Cancer-Project.mlflow',
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config
        

In [10]:

import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
from mlflow.models import infer_signature
import tensorflow as tf
import mlflow
import mlflow.tensorflow        
import dagshub
import mlflow
import os


In [11]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
        
    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)
        
    # def log_into_mlflow(self):
    #     mlflow.set_registry_uri(self.config.mlflow_uri)
    #     tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
    #     # Generate a sample input for the signature and example
    #     self._valid_generator()  # Ensure valid_generator is created
    #     sample_batch = next(iter(self.valid_generator))
    #     input_data = sample_batch[0]  # Example inputs
    #     predictions = self.model.predict(input_data)  # Example outputs

    #     # Infer the signature
    #     signature = infer_signature(input_data, predictions)
        
    #     # Define the input example
    #     input_example = input_data[:1]  # Single sample from batch

    #     with mlflow.start_run():
    #         mlflow.log_params(self.config.all_params)
    #         mlflow.log_metrics({"loss": self.score[0], "accuracy": self.score[1]})

    #         if tracking_url_type_store != "file":
    #             mlflow.keras.log_model(
    #                 self.model, 
    #                 "model", 
    #                 registered_model_name="VGG16Model", 
    #                 signature=signature,
    #                 input_example=input_example
    #             )
    #         else:
    #             mlflow.keras.log_model(
    #                 self.model, 
    #                 "model", 
    #                 signature=signature,
    #                 input_example=input_example
    #             )

    # def log_into_mlflow(self):
    #     mlflow.set_registry_uri(self.config.mlflow_uri)
    #     tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
    #     with mlflow.start_run():
    #         mlflow.log_params(self.config.all_params)
    #         mlflow.log_metrics(
    #             {"loss": self.score[0], "accuracy": self.score[1]}
    #         )
    #         # Model registry does not work with file store
    #         if tracking_url_type_store != "file":

    #             # Register the model
    #             # There are other ways to use the Model Registry, which depends on the use case,
    #             # please refer to the doc for more information:
    #             # https://mlflow.org/docs/latest/model-registry.html#api-workflow
    #             mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
    #         else:
    #             mlflow.keras.log_model(self.model, "model")
    
#     def log_into_mlflow(self):


# # Initialize DagsHub repository
#         dagshub.init(repo_owner='maushamkumar', repo_name='Chest-Cancer-Project', mlflow=True)

# # Set environment variables (if not already done)
# # os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/maushamkumar/Chest-Cancer-Project.mlflow' 

#         os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/maushamkumar/Chest-Cancer-Project.mlflow'
#         os.environ['MLFLOW_TRACKING_USERNAME'] = 'Mausham-Kumar'  # Your DagsHub username
#         os.environ['MLFLOW_TRACKING_PASSWORD'] = '408280058a024dea4a01aef21d922ac19e15bb2f'  # Access token

#         # Verify environment setup
#         print("MLFLOW_TRACKING_URI:", os.environ.get('MLFLOW_TRACKING_URI'))
#         print("MLFLOW_TRACKING_USERNAME:", os.environ.get('MLFLOW_TRACKING_USERNAME'))
#         print("MLFLOW_TRACKING_PASSWORD:", 
#               os.environ.get('MLFLOW_TRACKING_PASSWORD'))

#         mlflow.set_tracking_uri(os.environ['MLFLOW_TRACKING_URI'])
#         tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

#         with mlflow.start_run():
#             mlflow.log_params(self.config.all_params)
#             mlflow.log_metrics({"loss":
#                 self.score[0], "accuracy": self.score[1]})

#             if tracking_url_type_store != "file":
#                 mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
#             else:
#                 mlflow.keras.log_model(self.model, "model")

    def log_into_mlflow(self):
        # Initialize DagsHub repository
        dagshub.init(repo_owner='maushamkumar', repo_name='Chest-Cancer-Project', mlflow=True)

        # Set environment variables
        os.environ['MLFLOW_TRACKING_URI'] = self.config.mlflow_uri
        os.environ['MLFLOW_TRACKING_TOKEN'] = "840e97a11ba7812e407d7188fc60a602455a3cbb"
    
        # os.environ['MLFLOW_TRACKING_USERNAME'] = 'Mausham-Kumar'
        # os.environ['MLFLOW_TRACKING_PASSWORD'] = '840e97a11ba7812e407d7188fc60a602455a3cbb'

        # Print the environment variables for debugging
        print("MLFLOW_TRACKING_URI:", os.environ.get('MLFLOW_TRACKING_URI'))
        print("MLFLOW_TRACKING_USERNAME:", os.environ.get('MLFLOW_TRACKING_USERNAME'))
        print("MLFLOW_TRACKING_PASSWORD:", os.environ.get('MLFLOW_TRACKING_PASSWORD'))

        mlflow.set_tracking_uri(os.environ['MLFLOW_TRACKING_URI'])
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({"loss": self.score[0], "accuracy": self.score[1]})

            # Ensure the model is not None and is a Keras model
            if self.model is None or not isinstance(self.model, tf.keras.Model):
                print("Model is not loaded correctly.")
                return

            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


In [12]:
# try:
#     config = ConfigurationManager()
#     eval_config = config.get_evaluation_config()
#     evaluation = Evaluation(eval_config)
#     evaluation.evaluation()
#     evaluation.log_into_mlflow()

# except Exception as e:
#    raise e

try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()         # Runs the model evaluation
    evaluation.log_into_mlflow()    # Logs the results to MLflow
except Exception as e:
    raise e


[2024-10-27 17:40:48,316 :INFO: common: Reading yaml file: config/config.yml loaded successfully]
[2024-10-27 17:40:48,327 :INFO: common: Reading yaml file: params.yaml loaded successfully]
[2024-10-27 17:40:48,328 :INFO: common: created directory at: artifacts]
Found 102 images belonging to 2 classes.
7/7 [==============================] - 11s 2s/step - loss: 1.7844 - accuracy: 0.7255
[2024-10-27 17:40:59,957 :INFO: common: json file saved at: scores.json]
[2024-10-27 17:41:00,536 :INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 401 Unauthorized"]


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=ca82598c-3bf6-40b9-b6e4-405b1b030a9d&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=7e1afdc42c2aaaff3ccf7fa5b760e36a1b4a70961f486be69ecee8eec27a5e97




/Users/sanjaymahto/Chess-Cancer-Classification/venv/lib/python3.8/site-packages/rich/live.py:231: UserWarning: 
install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

[2024-10-27 17:41:09,713 :INFO: _client: HTTP Request: POST https://dagshub.com/login/oauth/middleman "HTTP/1.1 200 OK"]


...

Repository maushamkumar/Chest-Cancer-Project initialized!

[2024-10-27 17:41:12,082 :INFO: helpers: Repository maushamkumar/Chest-Cancer-Project initialized!]
MLFLOW_TRACKING_URI: https://dagshub.com/maushamkumar/Chest-Cancer-Project.mlflow
MLFLOW_TRACKING_USERNAME: 7da04c392c2d1af2257a14b3dd4918ebf83bb9dc
MLFLOW_TRACKING_PASSWORD: 7da04c392c2d1af2257a14b3dd4918ebf83bb9dc


2024/10/27 17:41:14 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


INFO:tensorflow:Assets written to: /var/folders/wg/8rqwg0nd7995t2tzgq40dsph0000gn/T/tmphbud5_1g/model/data/model/assets
[2024-10-27 17:41:15,358 :INFO: builder_impl: Assets written to: /var/folders/wg/8rqwg0nd7995t2tzgq40dsph0000gn/T/tmphbud5_1g/model/data/model/assets]


2024/10/27 17:41:20 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'VGG16Model' already exists. Creating a new version of this model...
2024/10/27 17:41:44 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 2
Created version '2' of model 'VGG16Model'.
2024/10/27 17:41:45 INFO mlflow.tracking._tracking_service.client: 🏃 View run grandiose-cow-987 at: https://dagshub.com/maushamkumar/Chest-Cancer-Project.mlflow/#/experiments/0/runs/2421d1cbd42b4327a1b526592107cada.
2024/10/27 17:41:45 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/maushamkumar/Chest-Cancer-Project.mlflow/#/experiments/0.


In [13]:
print((os.environ['MLFLOW_TRACKING_URI']))

https://dagshub.com/maushamkumar/Chest-Cancer-Project.mlflow


In [20]:
!git ls-remote https://dagshub.com/maushamkumar/Chest-Cancer-Project.mlflow


remote: 404 page not found
fatal: repository 'https://dagshub.com/maushamkumar/Chest-Cancer-Project.mlflow/' not found


In [21]:
!git ls-remote https://github.com/maushamkumar/Chest-Cancer-Project.git

18515bb6abbebfb6a3d328aca6b273c5cb161fe2	HEAD
18515bb6abbebfb6a3d328aca6b273c5cb161fe2	refs/heads/main


In [24]:
import os
from urllib.parse import urlparse
import mlflow
import dagshub

class YourClass:
    def log_into_mlflow(self):
        # Initialize DagsHub repository
        dagshub.init(repo_owner='maushamkumar', repo_name='Chest-Cancer-Project', mlflow=True)

        # Set environment variables
        os.environ['MLFLOW_TRACKING_URI'] = self.config.mlflow_uri
        os.environ['MLFLOW_TRACKING_TOKEN'] = "840e97a11ba7812e407d7188fc60a602455a3cbb"


        # Print the environment variables for debugging
        print("MLFLOW_TRACKING_URI:", os.environ.get('MLFLOW_TRACKING_URI'))
        print("MLFLOW_TRACKING_USERNAME:", os.environ.get('MLFLOW_TRACKING_USERNAME'))
        print("MLFLOW_TRACKING_PASSWORD:", os.environ.get('MLFLOW_TRACKING_PASSWORD'))

        # Set MLflow tracking URI
        mlflow.set_tracking_uri(os.environ['MLFLOW_TRACKING_URI'])
        
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        if tracking_url_type_store == "http" or tracking_url_type_store == "https":
            # If using HTTP(S), set the username and password for authentication
            mlflow.set_tracking_uri(f"http://{os.environ['MLFLOW_TRACKING_USERNAME']}:{os.environ['MLFLOW_TRACKING_PASSWORD']}@{self.config.mlflow_uri}")

In [18]:
mlflow.set_tracking_uri("https://dagshub.com/maushamkumar/Chest-Cancer-Project.mlflow")

In [13]:
import requests

response = requests.get(
    'https://dagshub.com/api/v1/user',
    auth=('token', '5fed9dba099f2504c17ba71e8f00ea56c4b5f830')
)

print(response.json())


{'message': 'Please authorize in order to access this resource'}


In [11]:
import os

# No need to set the environment variables again, as they're already set in the terminal
print(os.environ['MLFLOW_TRACKING_URI'])  # Just to verify it's accessible


KeyError: 'MLFLOW_TRACKING_URI'

In [12]:
import os

# Setting environment variables (if needed)
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/maushamkumar/Chest-Cancer-Project.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME'] = 'Mausham-Kumar'
os.environ['MLFLOW_TRACKING_PASSWORD'] = 'd9d8c948482916cbef26519e40e72a2888bacbc0'

# Verify environment variables
try:
    print("MLFLOW_TRACKING_URI:", os.environ['MLFLOW_TRACKING_URI'])
    print("MLFLOW_TRACKING_USERNAME:", os.environ['MLFLOW_TRACKING_USERNAME'])
    print("MLFLOW_TRACKING_PASSWORD:", os.environ['MLFLOW_TRACKING_PASSWORD'])
except KeyError as e:
    print(f"KeyError: {e} - Environment variable not set.")


MLFLOW_TRACKING_URI: https://dagshub.com/maushamkumar/Chest-Cancer-Project.mlflow
MLFLOW_TRACKING_USERNAME: Mausham-Kumar
MLFLOW_TRACKING_PASSWORD: d9d8c948482916cbef26519e40e72a2888bacbc0


In [ ]:
with mlflow.start_run():
    mlflow.log_params(self.config.all_params)
    mlflow.log_metrics({"loss": self.score[0], "accuracy": self.score[1]})
    mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")


In [13]:
import dagshub
import mlflow
import os

# Initialize DagsHub repository
dagshub.init(repo_owner='maushamkumar', repo_name='Chest-Cancer-Project', mlflow=True)

# Set environment variables (if not already done)
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/maushamkumar/Chest-Cancer-Project.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME'] = 'Mausham-Kumar'  # Your DagsHub username
os.environ['MLFLOW_TRACKING_PASSWORD'] = 'd9d8c948482916cbef26519e40e72a2888bacbc0'  # Your access token

# # Start an MLflow run
# with mlflow.start_run():
#     mlflow.log_param('parameter_name', 'value')  # Replace with your parameter
#     mlflow.log_metric('metric_name', 1)  # Replace with your metric

# print("Logging completed.")


[2024-10-27 13:47:02,182 :INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/user "HTTP/1.1 200 OK"]


Accessing as maushamkumar

[2024-10-27 13:47:02,188 :INFO: helpers: Accessing as maushamkumar]
[2024-10-27 13:47:02,756 :INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/maushamkumar/Chest-Cancer-Project "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "maushamkumar/Chest-Cancer-Project"

[2024-10-27 13:47:02,783 :INFO: helpers: Initialized MLflow to track repo "maushamkumar/Chest-Cancer-Project"]


Repository maushamkumar/Chest-Cancer-Project initialized!

[2024-10-27 13:47:02,784 :INFO: helpers: Repository maushamkumar/Chest-Cancer-Project initialized!]


In [21]:
import os

os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/maushamkumar/Chest-Cancer-Project.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME'] = 'Mausham-Kumar'  # Your DagsHub username
os.environ['MLFLOW_TRACKING_PASSWORD'] = 'd9d8c948482916cbef26519e40e72a2888bacbc0'  # Your access token


In [23]:
import dagshub
import mlflow

# Initialize DagsHub with MLflow
dagshub.init(repo_owner='maushamkumar', repo_name='Chest-Cancer-Project', mlflow=True)

# Start MLflow run
with mlflow.start_run():
    mlflow.log_param('parameter_name', 'value')  # Replace with your parameter
    mlflow.log_metric('metric_name', 1)  # Replace with your metric

print("Logging completed.")


[2024-10-27 15:06:33,642 :INFO: _client: HTTP Request: GET https://dagshub.com/api/v1/repos/maushamkumar/Chest-Cancer-Project "HTTP/1.1 200 OK"]


Initialized MLflow to track repo "maushamkumar/Chest-Cancer-Project"

[2024-10-27 15:06:33,652 :INFO: helpers: Initialized MLflow to track repo "maushamkumar/Chest-Cancer-Project"]


Repository maushamkumar/Chest-Cancer-Project initialized!

[2024-10-27 15:06:33,654 :INFO: helpers: Repository maushamkumar/Chest-Cancer-Project initialized!]


2024/10/27 15:06:35 INFO mlflow.tracking._tracking_service.client: 🏃 View run flawless-wren-363 at: https://dagshub.com/maushamkumar/Chest-Cancer-Project.mlflow/#/experiments/0/runs/c12ce3af701e4f16bf3b5a293c0e26b0.
2024/10/27 15:06:35 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/maushamkumar/Chest-Cancer-Project.mlflow/#/experiments/0.


Logging completed.


In [ ]:
#Kuchbhi@143
#DagsHub password : Kuchbhi@143

In [16]:
import os
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from dataclasses import dataclass
from urllib.parse import urlparse

# Step 1: Define the Evaluation Configuration
@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

# Step 2: Configuration Manager class to read config files and parameters
class ConfigurationManager:
    def __init__(self, config_filepath='config/config.yml', params_filepath='params.yaml'):
        self.config = read_yaml(config_filepath)  # replace with actual config read function
        self.params = read_yaml(params_filepath)  # replace with actual params read function

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model=Path("artifacts/training/model.h5"),
            training_data=Path("artifacts/data_ingestion/Chest-CT-Scan-data"),
            mlflow_uri="https://dagshub.com/maushamkumar/Chest-Cancer-Project.mlflow",
            all_params=self.params,
            params_image_size=self.params['IMAGE_SIZE'],
            params_batch_size=self.params['BATCH_SIZE']
        )
        return eval_config

# Step 3: Evaluation class for handling model evaluation and logging
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        data_gen_args = dict(rescale=1./255, validation_split=0.30)
        data_flow_args = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )
        valid_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(**data_gen_args)
        self.valid_generator = valid_data_gen.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **data_flow_args
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json("scores.json", scores)  # replace with actual save_json function

    def log_into_mlflow(self):
        os.environ['MLFLOW_TRACKING_URI'] = self.config.mlflow_uri
        os.environ['MLFLOW_TRACKING_USERNAME'] = 'Mausham-Kumar'  # DagsHub username
        os.environ['MLFLOW_TRACKING_PASSWORD'] = 'd9d8c948482916cbef26519e40e72a2888bacbc0'  # DagsHub token

        mlflow.set_tracking_uri(os.environ['MLFLOW_TRACKING_URI'])
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics({"loss": self.score[0], "accuracy": self.score[1]})

            if tracking_url_type_store != "file":
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

# Step 4: Main execution script
try:
    config_manager = ConfigurationManager()
    eval_config = config_manager.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()
    print("Evaluation and logging completed successfully.")
except Exception as e:
    raise e


EnsureError: Argument path_to_yaml of type <class 'str'> to <function read_yaml at 0x30f83d160> does not match annotation type <class 'pathlib.Path'>